In [19]:
import time
import pyautogui
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException


In [20]:
# Configuration
list_ids = [20240701, 20240702, 20240703, 20240704, 20240705]  # Edit your list IDs here
base_list_name = "july_24"  # Edit your base list name here
chromedriver_path = 'chromedriver.exe'
base_url = "https://pos.mscall.net/vicidial/admin.php?ADD=100"
base_file_path = r"E:\3.download_from_database_pos2\output\pos{}_{}.csv"  # Will be formatted with index


# Configure ChromeDriver
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)
driver.maximize_window()
wait = WebDriverWait(driver, 250)  # Increased timeout to 60 seconds

In [21]:
def wait_for_page_reload(previous_state=None):
    """Wait for page to fully reload by checking page state changes"""
    try:
        # First wait for document.readyState to become complete
        WebDriverWait(driver, 250).until(
            lambda d: d.execute_script("return document.readyState") == 'complete'
        )
        
        # Then wait for body element to be present and visible
        WebDriverWait(driver, 250).until(
            lambda d: d.find_element(By.TAG_NAME, 'body').is_displayed()
        )
        
        # Additional check for specific loader to disappear (if applicable)
        # WebDriverWait(driver, 60).until(
        #     EC.invisibility_of_element_located((By.ID, "loader"))
        # )
    except Exception as e:
        print(f"Page reload detection warning: {str(e)}")
        # Take screenshot for debugging
        driver.save_screenshot("page_reload_error.png")

In [22]:
def login():
    """Login to the system"""
    driver.get(base_url)
    
    # Wait for auth popup and enter credentials
    time.sleep(3)  # Necessary for auth popup to appear
    pyautogui.write("posds2")
    pyautogui.press("tab")
    pyautogui.write("8rA327l6ecf")
    pyautogui.press("enter")
    
    # Wait for main page to load
    wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'ADD=111')]")))

In [23]:
def process_list(list_id, index):
    """Process a single list"""
    try:
        print(f"Processing list ID: {list_id} - Part {index}")


        # Navigate to list creation page
        wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'ADD=111')]"))).click()
        wait_for_page_reload()
        time.sleep(2)

        # Fill list information
        list_name = f"{base_list_name}_p{index}" if len(list_ids) > 1 else base_list_name
        wait.until(EC.presence_of_element_located((By.NAME, "list_id"))).send_keys(str(list_id))
        driver.find_element(By.NAME, "list_name").send_keys(list_name)
        
        today_date = datetime.now().strftime("%Y-%m-%d")
        driver.find_element(By.NAME, "list_description").send_keys(f"filtered_{today_date}")
        
        # Select campaign
        driver.find_element(By.XPATH, "//select[@name='campaign_id']/option[@value='POS']").click()
        time.sleep(5)
        
        # Submit and wait for reload
        driver.find_element(By.XPATH, "//input[@value='SUBMIT']").click()
        wait_for_page_reload()
        time.sleep(2)

        # Go to list loader
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//a[contains(@href, 'admin_listloader_fourth_gen.php')]"))).click()
        wait_for_page_reload()
        time.sleep(2)

        # Upload file
        date_prefix = str(list_id)[:6]  # First 6 digits
        file_path = base_file_path.format(date_prefix, index)
        file_input = wait.until(EC.presence_of_element_located((By.NAME, "leadfile")))
        file_input.send_keys(file_path)
        
        # Wait for file processing
        list_value_xpath = f'//option[@value="\\\'{list_id}\\\'"]'
        wait.until(EC.element_to_be_clickable((By.XPATH, list_value_xpath))).click()
        
        # Select options
        driver.find_element(By.XPATH, '//select[@name="phone_code_override"]/option[contains(text(), "1 - USA")]').click()
        driver.find_element(By.XPATH, "//input[@name='file_layout' and @value='custom']").click()

        # Submit and wait for field mapping page
        driver.find_element(By.XPATH, "//input[@value='SUBMIT']").click()
        wait_for_page_reload()
        time.sleep(5)

        # Set field mappings
        field_mappings = {
            'phone_number_field': 'phone',
            'first_name_field': 'first_name',
            'last_name_field': 'last_name',
            'address1_field': 'address1',
            'address2_field': 'address2',
            'city_field': 'city',
            'state_field': 'state',
            'postal_code_field': 'postal_code',
            'date_of_birth_field': 'date_of_birth',
            'email_field': 'email',
            'comments_field': 'comments'
        }
        
        for field, text in field_mappings.items():
            xpath = f"//select[@name='{field}']/option[contains(text(), '{text}')]"
            wait.until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
            
        time.sleep(5)
        
        # Final processing
        driver.find_element(By.XPATH, "//input[@value='OK TO PROCESS']").click()
        wait_for_page_reload()
        time.sleep(5)

        # Return to admin page
        wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href,'admin.php?ADD=100')]"))).click()
        wait_for_page_reload()
        time.sleep(5)
        
        print(f"Successfully processed list ID: {list_id} ({list_name}) with file: {file_path}")
        
    except Exception as e:
        print(f"Error processing list ID {list_id}: {str(e)}")
        driver.save_screenshot(f"error_{list_id}_{index}.png")

In [24]:
# Main execution
if __name__ == "__main__":
    try:
        login()
        
        for index, list_id in enumerate(list_ids, start=1):
            process_list(list_id, index)
            
        print("All lists processed successfully!")
        
    except Exception as e:
        print(f"Critical error: {str(e)}")
        
    finally:
        driver.quit()

Processing list ID: 20240701 - Part 1
Page reload detection warning: HTTPConnectionPool(host='localhost', port=57568): Read timed out. (read timeout=120)
Successfully processed list ID: 20240701 (july_24_p1) with file: E:\3.download_from_database_pos2\output\pos202407_1.csv
Processing list ID: 20240702 - Part 2
Page reload detection warning: HTTPConnectionPool(host='localhost', port=57568): Read timed out. (read timeout=120)
Successfully processed list ID: 20240702 (july_24_p2) with file: E:\3.download_from_database_pos2\output\pos202407_2.csv
Processing list ID: 20240703 - Part 3
Page reload detection warning: HTTPConnectionPool(host='localhost', port=57568): Read timed out. (read timeout=120)
Successfully processed list ID: 20240703 (july_24_p3) with file: E:\3.download_from_database_pos2\output\pos202407_3.csv
Processing list ID: 20240704 - Part 4
Page reload detection warning: HTTPConnectionPool(host='localhost', port=57568): Read timed out. (read timeout=120)
Successfully processe